In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython import display
from matplotlib import pyplot as plt
%matplotlib inline

In [58]:
import pathlib

import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

from ikapati.data import io
from ikapati.visualization import visualize

In [4]:
species = "Apple"
data_dir = pathlib.Path("../data/processed")
train_dir = data_dir.joinpath(species)
model_dir = pathlib.Path("../models")
metadata_file_path = train_dir.joinpath("metadata.json")
base_dir = pathlib.Path.cwd()

print(f"species: {species} - train_dir: {str(train_dir)} - metadata_file_path: {str(metadata_file_path)}")

species: Apple - train_dir: ../data/processed/Apple - metadata_file_path: ../data/processed/Apple/metadata.json


In [5]:
metadata = io.read_metadata(str(metadata_file_path))
model_id = metadata["id"]

print(f"model_id: {model_id}")

model_id: 4347264e-426c-4a2f-8cbe-3ac61a421685


In [6]:
training_log_path = model_dir.joinpath(model_id, "training.log")
training_log = pd.read_csv(training_log_path)

In [7]:
model_dir_paths = training_log["model_dir_path"].values
model_dir_paths

array(['models/4347264e-426c-4a2f-8cbe-3ac61a421685/2020-02-03__03_4307',
       'models/4347264e-426c-4a2f-8cbe-3ac61a421685/2020-02-03__03_5230',
       'models/4347264e-426c-4a2f-8cbe-3ac61a421685/2020-02-03__03_5418',
       'models/4347264e-426c-4a2f-8cbe-3ac61a421685/2020-02-03__03_5605'],
      dtype=object)

In [8]:
training_log

,start_time,end_time,activation,model_dir_path,learning_rate,dropout,epochs,batch_size
0,2020-02-03__03_4307,2020-02-03__03_4436,relu,models/4347264e-426c-4a2f-8cbe-3ac61a421685/20...,0.100,False,20,64
1,2020-02-03__03_5230,2020-02-03__03_5357,relu,models/4347264e-426c-4a2f-8cbe-3ac61a421685/20...,0.010,False,20,64
2,2020-02-03__03_5418,2020-02-03__03_5545,relu,models/4347264e-426c-4a2f-8cbe-3ac61a421685/20...,0.001,False,20,64
3,2020-02-03__03_5605,2020-02-03__03_5734,relu,models/4347264e-426c-4a2f-8cbe-3ac61a421685/20...,0.001,True,20,64


# Evaluation

In [9]:
def run_evaluation(log_row):
    model_dir_path = log_row.model_dir_path
    learning_rate = log_row.learning_rate
    activation = log_row.activation
    model_metadata_file_path = pathlib.Path("..", model_dir_path, "metadata.json")
    model_metadata = io.read_metadata(str(model_metadata_file_path))
    print(
        "batch_size:", model_metadata["arguments"]["batch_size"], 
        "activation:", model_metadata["arguments"]["epochs"],
        "architecture:", model_metadata["arguments"]["architecture"],
        "dropout:", model_metadata["arguments"]["dropout"],
    )
    dropout = "-dropout" if model_metadata["arguments"]["dropout"] else ""
    history = model_metadata["history"]
    metrics_df = visualize.create_metrics_dataframe(history)
    loss_plot = visualize.learning_curves("loss", metrics_df)
    visualize.save_plot(loss_plot, f"../reports/figures/{species}-{activation}{dropout}_{learning_rate}.loss.svg")
    plt.clf()
    accuracy_plot = visualize.learning_curves("accuracy", metrics_df)
    visualize.save_plot(accuracy_plot, f"../reports/figures/{species}-{activation}{dropout}_{learning_rate}.accuracy.svg")
    plt.clf()

In [10]:
for index, log_row in training_log.iterrows():
    run_evaluation(log_row)

batch_size: 64 activation: 20 architecture: alexnet dropout: False
batch_size: 64 activation: 20 architecture: alexnet dropout: False
batch_size: 64 activation: 20 architecture: alexnet dropout: False
batch_size: 64 activation: 20 architecture: alexnet dropout: True


<Figure size 432x288 with 0 Axes>

# Test Dataset

In [60]:
def evaluate_on_test_data(model_dir_path, batch_size=32):
    model_path = pathlib.Path("..", model_dir_path, "final.h5")
    model_metadata_file_path = pathlib.Path("..", model_dir_path, "metadata.json")
    model_metadata = io.read_metadata(str(model_metadata_file_path))
    print(
        "batch_size:", model_metadata["arguments"]["batch_size"], 
        "activation:", model_metadata["arguments"]["epochs"],
        "architecture:", model_metadata["arguments"]["architecture"],
        "dropout:", model_metadata["arguments"]["dropout"],
    )
    model = tf.keras.models.load_model(model_path)
    test_data_path = train_dir.joinpath("test.tfrecord")
    test_dataset = io.read_dataset(str(test_data_path), batch_size, metadata["num_classes"])
    steps = metadata["file_counts"]["test"] // batch_size
    model.evaluate(test_dataset, steps=steps)

In [61]:
for index, log_row in training_log.iterrows():
    evaluate_on_test_data(log_row.model_dir_path)

batch_size: 64 activation: 20 architecture: alexnet dropout: False
9/9 [==============================] - 1s 57ms/step - loss: 1.3033 - accuracy: 0.5417
batch_size: 64 activation: 20 architecture: alexnet dropout: False
9/9 [==============================] - 1s 56ms/step - loss: 1.1683 - accuracy: 0.5486
batch_size: 64 activation: 20 architecture: alexnet dropout: False
9/9 [==============================] - 1s 57ms/step - loss: 0.2791 - accuracy: 0.9340
batch_size: 64 activation: 20 architecture: alexnet dropout: True
9/9 [==============================] - 0s 51ms/step - loss: 0.3911 - accuracy: 0.9097
